In [11]:
import os
import json
import requests
from dotenv import load_dotenv


In [21]:
load_dotenv(os.path.join(os.path.expanduser('~'), '.ncats_llama.env'))


True

In [25]:
URI = os.environ.get('LLAMA_URI')


In [26]:

def generate_with_llama(prompt):
    api_key = os.environ.get('LLAMA_KEY')

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 256,
        "stream": True
    }

    with requests.post(URI, headers=headers, json=data, stream=True) as response:
        if response.status_code == 200:
            for line in response.iter_lines():
                if line:
                    try:
                        json_str = line.decode('utf-8')
                        if json_str.startswith('data: '):
                            json_str = json_str[len('data: '):]
                        chunk = json.loads(json_str)

                        choices = chunk.get('choices', [])
                        if choices:
                            delta = choices[0].get('delta', {})
                            content = delta.get('content', '')
                            if content:
                                yield content
                    except (json.JSONDecodeError, IndexError, KeyError):
                        continue
        else:
            raise Exception(f"Request failed with status code {response.status_code}: {response.text}")


for content in generate_with_llama("Tell me two joke."):
    print(content, end='', flush=True)

Here are two jokes for you:

**Joke 1:**
Why couldn't the bicycle stand up by itself?
Because it was two-tired!

**Joke 2:**
What do you call a fake noodle?
An impasta!

I hope you found these jokes amusing!